### Main notebook

In [1]:
import pandas as pd
import numpy as np
import json
import networkx as nx
from tqdm import tqdm

# Add path 
import sys
sys.path.append('../')
from compound import Compound
from reaction import Reaction
from graph import Graph
from data import Data

# suppres rdkit warnings
import rdkit
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

# read data from csv
cpds = pd.read_csv('../data/compounds_final.csv', index_col=0) # containing toxicity
rxns = pd.read_csv('../data/reactions_final.csv', index_col=0)
pairs = pd.read_csv('../data/pairs_final_RPAIRS_pred.csv', index_col=0)
cofactors = pd.read_csv('../data/original/cofactors_KEGG.csv')

# create class instances
data = Data()
graph = Graph(pairs=pairs)

# Create a Compound object for each row in the DataFrame and add it to the data
for index, row in cpds.iterrows():
    entry = row['Entry']
    name = row['Names']
    formula = row['Formula']
    mw = row['mol_weight']
    smiles = row['SMILES']
    is_cofactor = row['Entry'] in cofactors['Entry'].values
    # is_toxic = row['toxic']
    is_polymer = row['polymer']

    compound = Compound(entry, name, formula, mw, smiles, is_cofactor, is_polymer)
    data.add_element('compound', compound)

# Create a Reaction object for each row in the DataFrame and add it to the data
for index, row in rxns.iterrows():
    entry = row['Entry']
    name = row['Names']
    compounds = row['Compound']
    enzyme = row['EC Number']

    reaction = Reaction(entry, name, compounds, enzyme)
    data.add_element('reaction', reaction)

In [2]:
# CREATE GRAPH
graph.create_graph(data=data, pairs=pairs)

graph.calculate_edge_mol_weight(data)
graph.calculate_smiles_similarity(data)

# nodes: 7997 
# edges: 11783


100%|██████████| 11783/11783 [00:25<00:00, 461.62it/s]


In [3]:
######### VALIDATION SET FROM nicepath ###########
test_cases = pd.read_csv('../data/original/test_cases.csv')
test_cases['source'] = test_cases['Pathway '].apply(lambda x: x.split(',')[0])
test_cases['target'] = test_cases['Pathway '].apply(lambda x: x.split(',')[len(x.split(','))-1])
test_cases['paths_list'] = test_cases['Pathway '].apply(lambda x: x.split(','))

''' Get just a single pathway from test_cases for testing '''
# test_cases = test_cases.iloc[11:1]

paths = graph.validate(test_cases, 'mol_weight')

  0%|          | 0/50 [00:00<?, ?it/s]

Searching for pathway between C00082 and C01533


  2%|▏         | 1/50 [00:00<00:25,  1.93it/s]

Searching for pathway between C00223 and C00761


  4%|▍         | 2/50 [00:00<00:21,  2.28it/s]

...

Searching for pathway between C11249 and C02232


 96%|█████████▌| 48/50 [00:17<00:01,  1.99it/s]

Searching for pathway between C00829 and C00805


100%|██████████| 50/50 [00:17<00:00,  2.84it/s]

Searching for pathway between C07535 and C18283
Correct pathway predictions: 23
Correct pathway predictions (%): 46.0


### Study graph

### Try cluster graph nodes

### Try similarity based on SMILES

In [4]:
smiles1 = data.get_compound_by_id('C00223').smiles
smiles2 = data.get_compound_by_id('C00323').smiles

from rdkit import Chem
from rdkit import DataStructs

ms = [Chem.MolFromSmiles(smiles1), Chem.MolFromSmiles(smiles2)]
fs = [Chem.RDKFingerprint(x) for x in ms]
s = DataStructs.FingerprintSimilarity(fs[0], fs[1])
print('Similarity: '+str(s))


smiles1 = data.get_compound_by_id('C00223').smiles
smiles2 = data.get_compound_by_id('C12096').smiles

from rdkit import Chem
from rdkit import DataStructs

ms = [Chem.MolFromSmiles(smiles1), Chem.MolFromSmiles(smiles2)]
fs = [Chem.RDKFingerprint(x) for x in ms]
s = DataStructs.FingerprintSimilarity(fs[0], fs[1])
print('Similarity: '+str(s))

Similarity: 0.989145183175034
Similarity: 0.5102379634753735
